<a href="https://colab.research.google.com/github/neel26desai/news_summarization/blob/main/SummarizationHuggingFace_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sacremoses==0.0.53
!pip install datasets
!pip install transformers
!pip install transformers[torch]


In [ ]:
!pip install accelerate -U
! pip install evaluate rouge_score

In [ ]:
from datasets import load_dataset
from transformers import pipeline

In [ ]:
#loading the dataset
xsum_dataset = load_dataset(
    "xsum",
    version="1.2.0",
    cache_dir='/content/drive/MyDrive/Hugging_Face/data'
)  # Note: We specify cache_dir to use predownloaded data.
xsum_dataset
# The printed representation of this object shows the `num_rows`
# of each dataset split

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [ ]:
import pickle
#save the dataset as a pickle file
with open('/content/drive/MyDrive/xsum.pkl', 'wb') as f:
    pickle.dump(xsum_dataset, f)

In [ ]:
#we have save the data
import pickle
with open('/content/drive/MyDrive/xsum.pkl','rb') as f:
  xsum_dataset = pickle.load(f)


In [ ]:
from transformers import AutoTokenizer
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=250, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_xsum = xsum_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/204045 [00:00<?, ? examples/s]

Map:   0%|          | 0/11332 [00:00<?, ? examples/s]

Map:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [ ]:
# Save the dataset dict
with open('/content/drive/MyDrive/tokenized.pkl', 'wb') as f:
    pickle.dump(tokenized_xsum, f)

In [ ]:
#as we have already created the tokenized version' we'll simply load it
with open('/content/drive/MyDrive/tokenized.pkl','rb') as f:
  tokenized_xsum = pickle.load(f)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir='/content/drive/MyDrive/fine_tuned_t5/'+"my_awesome_xsum_model/results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_xsum["train"],
    eval_dataset=tokenized_xsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.706800,2.482651,0.286200,0.079300,0.225000,0.225000,18.826900
2,2.646000,2.437687,0.291500,0.082800,0.229200,0.229200,18.836900
3,2.618500,2.417630,0.294900,0.085200,0.232000,0.232000,18.836400
4,2.605900,2.411622,0.296300,0.086000,0.233100,0.233100,18.835900


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=25508, training_loss=2.671520611363854, metrics={'train_runtime': 8843.375, 'train_samples_per_second': 92.293, 'train_steps_per_second': 2.884, 'total_flos': 2.2051178810022298e+17, 'train_loss': 2.671520611363854, 'epoch': 4.0})

In [ ]:
model_dir = '/content/drive/MyDrive/fine_tuned_t5/model'
trainer.save_model(model_dir)

In [ ]:
new_model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

In [ ]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

In [ ]:
tokernizer2 = AutoTokenizer.from_pretrained(model_dir)

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model=new_model,tokenizer=tokernizer2)
summarizer("summarize:" +xsum_dataset['train'][0]['document'])

Token indices sequence length is longer than the specified maximum sequence length for this model (543 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'A flood warning has been put in place in Dumfries and the Nith after a torrential rain hit the town of Peebles.'}]

In [ ]:
"summarize:" +xsum_dataset['train'][0]['document']

'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but

In [ ]:
#uplading the new model to hugging face
from huggingface_hub import notebook_login

notebook_login()
ft_t5.push_to_hub('neel26d/newstuned_t5_summarizer')